In [3]:
from PyPDF2 import PdfReader

# Step 1: Extract text from the PDF
pdf_path = "Q3-2024-Press-Release-English.pdf"

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Extract text
raw_text = extract_text_from_pdf(pdf_path)

# Save the raw text for review
with open("extracted_text.txt", "w") as f:
    f.write(raw_text)

print("Text extraction complete. The raw text has been saved for review.")


Text extraction complete. The raw text has been saved for review.


In [4]:
import re
import json

# Load the raw text
with open("extracted_text.txt", "r") as f:
    raw_text = f.read()

# Step 1: Split the text into sections based on headings
def split_into_sections(text):
    # Match uppercase headings or headings followed by lines
    headings = re.findall(r"^[A-Z\s\-]+(?=\n)", text, re.MULTILINE)
    sections = re.split(r"^[A-Z\s\-]+(?=\n)", text, re.MULTILINE)
    
    # Combine headings and sections
    structured_data = [{"title": title.strip(), "content": section.strip()} 
                       for title, section in zip(headings, sections[1:])]
    return structured_data

# Step 2: Clean and preprocess sections
def clean_section(section):
    # Remove table-like lines
    section = re.sub(r"\s{2,}", " ", section)  # Replace extra spaces
    section = re.sub(r"[\d]+\s+[A-Za-z]+\s+[A-Za-z]+\s+", "", section)  # Remove table rows
    return section.strip()

structured_sections = split_into_sections(raw_text)
for section in structured_sections:
    section["content"] = clean_section(section["content"])

# Step 3: Save structured data for review
output_path = "structured_text.json"
with open(output_path, "w") as f:
    json.dump(structured_sections, f, indent=4)

output_path


'structured_text.json'

In [5]:
import re
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

# Load the structured JSON file
input_json_path = "structured_text.json"
output_cleaned_path = "cleaned_text.json"

with open(input_json_path, "r") as f:
    structured_data = json.load(f)

# Initialize preprocessing tools
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r"[^A-Za-z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

# Clean all sections
for section in structured_data:
    section["cleaned_content"] = clean_text(section["content"])

# Save the cleaned data
with open(output_cleaned_path, "w") as f:
    json.dump(structured_data, f, indent=4)

output_cleaned_path



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andressalguero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andressalguero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andressalguero/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'cleaned_text.json'

In [22]:
from collections import Counter
import re
import json

# Load the cleaned text JSON file
input_json_path = "cleaned_text.json"
output_qa_path = "qa_dataset_advanced.json"

with open(input_json_path, "r") as f:
    cleaned_data = json.load(f)

qa_dataset = []

# Function to derive frequent words from the text
def get_frequent_words(text, top_n=10):
    words = re.findall(r'\b\w+\b', text.lower())  # Extract words
    stop_words = {"the", "and", "of", "in", "to", "for", "a", "is", "on", "with", "by", "was", "at", "as", "it"}
    filtered_words = [word for word in words if word not in stop_words]
    word_counts = Counter(filtered_words)
    return [word for word, count in word_counts.most_common(top_n)]

# Function to derive questions based on patterns
def derive_pattern_based_questions(context):
    questions = []
    if "revenue" in context.lower():
        questions.append("What is the revenue?")
    if "%" in context:
        questions.append("What percentage changes are mentioned?")
    if "profit" in context.lower():
        questions.append("What is the profit mentioned in this section?")
    return questions

# Function to derive questions and answers
def derive_questions_and_answers(section):
    questions_answers = []
    context = section["cleaned_content"]

    # Generic Questions
    generic_questions = [
        "What is this section about?",
        "What is the main highlight?",
        "What improvements are suggested?",
        "What achievements are mentioned?",
    ]
    for question in generic_questions:
        questions_answers.append({
            "question": question,
            "answers": {
                "text": [context[:100]],  # Use the first 100 characters as a brief summary
                "answer_start": [0]
            }
        })

    # Specific Questions Based on Frequent Words
    frequent_words = get_frequent_words(context)
    for word in frequent_words:
        if word in context:
            questions_answers.append({
                "question": f"What is mentioned about '{word}'?",
                "answers": {
                    "text": [context[context.find(word):context.find(word) + 50]],
                    "answer_start": [context.find(word)]
                }
            })

    # Questions Based on Patterns
    pattern_questions = derive_pattern_based_questions(context)
    for question in pattern_questions:
        # Use the first 100 characters as a general answer for pattern-based questions
        questions_answers.append({
            "question": question,
            "answers": {
                "text": [context[:100]],
                "answer_start": [0]
            }
        })

    return context, questions_answers

# Process each section to derive QA pairs
for section in cleaned_data:
    context, questions_answers = derive_questions_and_answers(section)
    for qa in questions_answers:
        qa_dataset.append({
            "context": context,
            "question": qa["question"],
            "answers": qa["answers"]
        })

# Save the QA dataset to a JSON file
with open(output_qa_path, "w") as f:
    json.dump(qa_dataset, f, indent=4)

print(f"QA dataset saved to {output_qa_path}")



QA dataset saved to qa_dataset_advanced.json


In [23]:
# Load the QA dataset
input_json_path = "qa_dataset_advanced.json"
output_json_path = "qa_dataset_refined.json"

with open(input_json_path, "r") as f:
    qa_dataset = json.load(f)

refined_dataset = []

# Function to derive question variations
def generate_question_variations(base_question, keywords):
    variations = []
    for keyword in keywords:
        variations.append(base_question.replace("this section", f"the {keyword}"))
    return variations

# Function to refine answers
def refine_answer(context, answer_text, start_pos):
    # Ensure the answer is well-contained and concise
    if start_pos < 0 or start_pos >= len(context):
        return None, None  # Invalid position
    answer_end = start_pos + len(answer_text)
    if answer_end > len(context):
        return None, None
    return context[start_pos:answer_end], start_pos

# Function to extract key entities or patterns
def extract_entities(context):
    entities = []
    # Extract dates
    entities += re.findall(r'\b\d{4}\b', context)  # Years
    # Extract percentages
    entities += re.findall(r'\b\d+%|\bpercent\b', context.lower())
    # Extract monetary values
    entities += re.findall(r'\$\d+[,\d]*', context)
    return list(set(entities))  # Unique entities

# Refine the dataset
for qa_entry in qa_dataset:
    context = qa_entry["context"]
    question = qa_entry["question"]
    answers = qa_entry["answers"]

    # Refine answers
    refined_answer_text, refined_start_pos = refine_answer(
        context, answers["text"][0], answers["answer_start"][0]
    )
    if refined_answer_text is None:
        continue  # Skip invalid answers

    # Generate additional questions based on entities
    entities = extract_entities(context)
    entity_questions = generate_question_variations(
        "What is mentioned about this section?", entities
    )

    # Add original QA pair
    refined_dataset.append({
        "context": context,
        "question": question,
        "answers": {
            "text": [refined_answer_text],
            "answer_start": [refined_start_pos]
        }
    })

    # Add entity-based QA pairs
    for entity_question in entity_questions:
        refined_dataset.append({
            "context": context,
            "question": entity_question,
            "answers": {
                "text": [refined_answer_text],
                "answer_start": [refined_start_pos]
            }
        })

# Save the refined dataset
with open(output_json_path, "w") as f:
    json.dump(refined_dataset, f, indent=4)

print(f"Refined QA dataset saved to {output_json_path}")


Refined QA dataset saved to qa_dataset_refined.json


Fine-Tuning

In [12]:
from datasets import Dataset
import json

In [10]:

from transformers import BertTokenizerFast, BertForQuestionAnswering, Trainer, TrainingArguments
import torch

In [13]:
# Load the QA Dataset
with open("qa_dataset_refined.json", "r") as f:
    qa_data = json.load(f)

# Convert the JSON dataset into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "context": [entry["context"] for entry in qa_data],
    "question": [entry["question"] for entry in qa_data],
    "answers": [entry["answers"] for entry in qa_data],
})

In [14]:
# Split the dataset into training and validation sets
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [15]:
# Load the pretrained BERT tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Preprocessing function for tokenization
def preprocess_data(example):
    tokenized = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True,
    )
    start_char = example["answers"]["answer_start"][0]
    end_char = start_char + len(example["answers"]["text"][0])
    offsets = tokenized["offset_mapping"]

    # Find the start and end token indices
    start_token_idx = end_token_idx = 0
    for idx, (start, end) in enumerate(offsets):
        if start <= start_char < end:
            start_token_idx = idx
        if start < end_char <= end:
            end_token_idx = idx

    tokenized["start_positions"] = start_token_idx
    tokenized["end_positions"] = end_token_idx
    tokenized.pop("offset_mapping")  # Remove offset mapping as it's not needed for training

    return tokenized

In [17]:
# Preprocess the datasets
train_dataset = train_dataset.map(preprocess_data, batched=False)
eval_dataset = eval_dataset.map(preprocess_data, batched=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [18]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./bert-fine-tuned-qa",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
)

/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

/var/folders/8b/gvyhprgn3t764q19dkq1gt2c0000gp/T/ipykernel_92331/2976303973.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# Train the model
trainer.train()

  0%|          | 0/6 [00:00<?, ?it/s]

{'train_runtime': 8.6749, 'train_samples_per_second': 4.842, 'train_steps_per_second': 0.692, 'train_loss': 5.502456029256185, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=5.502456029256185, metrics={'train_runtime': 8.6749, 'train_samples_per_second': 4.842, 'train_steps_per_second': 0.692, 'total_flos': 10974463782912.0, 'train_loss': 5.502456029256185, 'epoch': 3.0})

In [21]:
model.save_pretrained("./bert-fine-tuned-qa")
tokenizer.save_pretrained("./bert-fine-tuned-qa")

('./bert-fine-tuned-qa/tokenizer_config.json',
 './bert-fine-tuned-qa/special_tokens_map.json',
 './bert-fine-tuned-qa/vocab.txt',
 './bert-fine-tuned-qa/added_tokens.json',
 './bert-fine-tuned-qa/tokenizer.json')

In [6]:
# re-import libraries to run a test only required if you are running the code in a different session
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch

In [7]:
# Load the fine-tuned model and tokenizer
model_path = "./bert-fine-tuned-qa"
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertForQuestionAnswering.from_pretrained(model_path)

In [3]:
# Check the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [8]:
def answer_question(context, question):
    # Tokenize input question and context
    inputs = tokenizer.encode_plus(
        question, context, return_tensors="pt", truncation=True, max_length=512
    )
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)

    # Get logits
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Identify the most likely start and end tokens
    start_idx = torch.argmax(start_scores).item()
    end_idx = torch.argmax(end_scores).item()

    # Validate start and end indices
    if start_idx > end_idx:
        print("Invalid span detected: Adjusting indices.")
        # Fallback: Choose tokens around the highest confidence start index
        end_idx = min(start_idx + 5, len(input_ids) - 1)

    # Decode tokens into the answer
    answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[start_idx:end_idx + 1])
    answer = tokenizer.convert_tokens_to_string(answer_tokens).strip()

    # Ensure answer is valid
    if not answer or "[CLS]" in answer or "[SEP]" in answer:
        answer = "Unable to extract a confident answer."

    return answer




In [5]:
# Example usage in a Jupyter cell
Context= "Retail sales increased by 15% this quarter. The company highlighted the success of its loyalty program."
Question= "What was the retail sales growth this quarter?"


# Get the answer
answer = answer_question(Context, Question)
print(f"Question: {Question}")
print(f"Answer: {answer}")

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [9]:
Context= "canadian tire corporation report third quarter result announces annual dividend increase th consecutive year share repurchase intention toronto november canadian tire corporation limited tsxctc tsx ctca ctc company today released third quarter result period ended september consolidated comparable salescompared q consolidated comparable sale compared q diluted normalized earnings per share eps compared q normalized basis annualized dividend increased per share alongside intention repurchase class non voting share delivered strong retail profitability third consecutive quarter sale trend improved said greg hick president ceo canadian tire corporation customer spending still constrained canadian seeking value finding triangle reward loyalty member earned redeemed u higher level quarter continue control cost manage margin carefully order balance lingering consumer economic headwind time investment made last two year position u well better omnichannel experience higher customer satisfaction score positive reaction new product hit shelf third quarter highlight consolidated comparable sale sportchek grew first quarter since q partially offset decline canadian tire retail ctr mark ctr comparable sale compared q customer continued prioritize essential category including automotive continued perform well strong quarter q led growth automotive service sportchek comparable sale marking two consecutive quarter sportchek outperformed industry trend targeted promotional event experience continued focus contributed growth athletic footwear hockey category mark comparable sale led industrial wear decline partially offset growth men short tshirts childrens wear top performer result ongoing strategic rollout category select mark store increased l oyalty engagement saw ctive registered loyalty member member took advantage offer engaged mass triangle promotion scanned loyalty card instore net promoter score np across company banner including ctr store investment focus strong stock availability key brand continued dr ive improvement positive customer sentiment improved retail profitability led higher consolidated income income tax ibt million increase normalized basisthe prior year retail ibt million normalized basis strong retail gross margin ratesolid cost control offset decline retail revenue ibt also benefited higher income equated around eps level result property sale gain insurance recovery financial service ibt quarter prior year higher net write offs operating expense partially offset higher revenue cardholder engagement remained strong gross average account receivable gaar mainly result higher average account balance ctc continues make solid progress key area within better connected strategy enhance customer experience drive efficiency almost capital invested since accomplishment third quarter included richer store digital customer experience store investment proceeding pace four new party city store added q refresh project expected completed end taking total since year end canadian tire expected deployed technology enhancement electronic shelf label locker ctc enhanced broadband capability supply chain productivity previously announced supply chain investment consolidation improving productivity saving including increased throughput result good toperson automation fully operational company calgary montreal distribution centre dc last stage planned supply chain investment include phased rollout ctc new transportation management system new vancouver dc set open continued margin accretion owned brand success continued strength category like automotive hockey contributing margin accretion owned brand penetrationdespite pressure discretionary category pipeline innovative quality owned brand product set roll consolidated overview revenue million compared period last year revenue excluding petroleum million decrease compared prior year consolidated income income tax million prior year normalized basis consolidated income income tax million diluted eps compared normalized basis prior year refer company q mda section normalizing item additional detail event impacted company quarter retail segment overview retail sale million compared third quarter retail sale excluding petroleumcomparable sale respectively ctr retail sale comparable sale period last year sportchek retail sale increased period last year comparable sale mark retail sale decreased period last year comparable sale helly hansen revenue compared period mainly due shift timing shipment wholesale customer retail revenue million decrease million compared prior year retail revenue excluding petroleum retail gross margin million compared third quarter prior year excluding petroleum retail gross margin rate excluding petroleum increased retail ibt q compared normalized basis prior year retail return invested capital roica trailing twelve month basis end third quarter compared end third quarter due decrease earnings prior period refer company q mda section normalizing item additional detail event impacted retail segment quarter financial service overview financial service segment income income tax quarter prior year higher net write offs operating expense partially offset higher revenue cardholder engag ement remained strong gaar relative prior year driven growth average account balance refer company q mda section detail event impacted financial service segment quarter ct reit overview diluted adjusted fund operation affo per unit compared q diluted net income per unit compared q announced three new investment totalling million expected add approximately incremental gross leasable area upon completion information refer q earnings release issued november capital expenditure total capital expenditure quarter compared q year todate basis operating capital expenditure quarter compared q full year expenditure expected company previously disclosed range million expenditure expected range million quarterly dividend company increased annual dividend th consecutive year voting class non voting share share increase approximately last year november company board director declared dividend payable march shareholder record january dividend considered eligible dividend tax purpose share repurchase november company announced intention repurchase class non voting share excess amount required anti dilutive purpose repurchase class non voting share made company existing normal course issuer bid ncib expires march thereafter renewed ncib subject regulatory approval non gaap financial measure ratio supplementary financial measure press release contains non gaap financial measure ratio supplementary financial measure reference q mda mean company management discussion analysis third quarter ended september available sedar httpwwwsedarplusca incorporated reference herein non gaap measure non gaap ratio standardized meaning gaap may comparable similar measure company nongaap financial measure ratio normalized diluted earnings per share normalized diluted eps non gaap ratio calculated dividing normalized net income attributable shareholder non gaap financial measure total diluted share company information measure see section company q mda following table reconciliation normalized net income attributable shareholder company respective gaap measure ytd ytd c million q q q q attributable shareholder item dc fire expense recovery gsthst related charge fair value redeemable financial instrument income income attributable shareholder eps non controlling interest included sum normalized net income attributable shareholder consolidated normalized income income tax retail normalized income income tax financial service normalized income income tax consolidated normalized income income tax retail normalized income income tax financial service normalized income income tax non gaap financial measure information measure see section com panys q mda table reconciles consolidated normalized income income tax income income tax ytd ytd c million q q q q income tax item dc fire expense recovery gsthst related charge fair value redeemable financial instrument income tax table reconciles retail normalized income income tax income income tax ytd ytd c million q q q q income tax le operating segment income tax item dc fire expense recovery income income tax table reconciles financial service normalized income income tax income income tax gaap measure reported consolidated financial statement ytd ytd c million q q q q income tax le operating segment income income tax item gsthst related charge normalized income income tax adjusted fund operation affo per unit affo per unit non gaap ratio calculated dividing affo weighted average number unit outstanding diluted basis affo non gaap financial measure following table reconciles gaap income income tax ffo reconciles ffo affo ytd ytd c million q q q q income tax le operating segment income income tax add ct reit fair value loss gain adjustment deferred tax lease principal payment right ofuse asset ct reit fair value equity award ct reit internal leasing expense fund operation le ct reit property straight line rent revenue ct reit direct leasing cost capital expenditure reserve adjusted fund operation invested capital roic roic calculated retail return divided retail invested capital retail return defined trailing annual retail tax earnings excluding interest expense lease related depreciation expense inter segment earnings normalizing em retail invested capital defined retail segment total asset le retail segment trade payable accrued liability inter segment balance based average trailing four quarter retail return retail invested capital non gaap financial measure information measure see section company q mda rolling c million q q income tax le operating segment income tax item operational efficiency program reduction related charge dc fire expense recovery income income tax le retail intercompany adjustment add retail interest expense right ofuse asset tax rate add retail tax retail return asset le average asset operating segment asset le average retail intercompany adjustment trade payable accrued liability trust asset invested capital include intercompany income received ct reit included retail segment inter company investment made retail segment ct reit ctfs trust accrued liability include trade payable short term derivative liability short term provision income tax payable expenditure operating capital expenditure non gaap financial measure information measure see section company q mda following table reconciles total addition investing activity reported consolidated statement cash flow operating capital expenditure ytd ytd c million q q q q total addition add accrued addition le ct reit acquisition development excluding vend in ctc expenditure appears consolidated statement cash flow investing activity b supplementary financial measure ratio measure supplementary financial measure see section supplementary financial measure company q mda information composition measure consolidated retail sale consolidated comparable sale revenue excluding petroleum retail revenue excluding petroleum retail sale retail sale excluding petroleum canadian tire retail comparable retail sale sportchek comparable retail sale mark comparable retail sale retail gross margin rate retail gross margin rate excluding petroleum gross average account receivables average account balance owned brand penetration forward looking statement press release contains information may constitute forward looking information within meaning applicable security law forward looking information provides insight regarding management current expectation plan allows investor others better understand company anticipated financial position result operation operating environment reader cautioned information may appropriate purpose although company belief forward looking information press release based information assumption belief current reasonable complete information necessarily subject number business economic competitive risk factor could cause actual result differ materially managem ents expectation plan set forth forward looking information company provide assurance financial operational performance plan aspiration forecast actually achieved achieved result increase company share price information material risk factor uncertainty material factor assumption applied preparing forward looking information could cause company actual result differ mater ially prediction forecast projection expectation conclusion refer section forward looking information investor communication company q mda well ctc public filing available httpswwwsedarplusca httpsinvestorscanadiantireca company undertake update forward looking information whether written oral may made time time behalf reflect new information future event otherwise except required applicable security law conference call canadian tire conduct conference call discus information included news release related matter et thursday november conference call available simultaneously entirety interested investor news medium webcast httpsinvestorscanadiantireca available replay website month canadian tire corporation canadian tire corporation limited tsx ctca tsx ctc ctc group company includes retail segment financial service division ct reit retail business led canadian tire founded c anadians product life canada across living playing fixing automotive seasonal gardening division party city partsource gas key part canadian tire network retail segment also includes mark leading sour ce casual industrial wear pro hockey life hockey specialty store catering elite player sportchek hockey expert sport expert atmosphere offer best active wear brand company close gasoline outlet supported strengthened ctc financial service division ten thousand people employed across canada around world ctc local dealer franchisees petroleum retailer addition ctc owns operates helly hansen leading technical outdoor brand based oslo norway information visit corpcanadiantireca information medium stephanie nadalin stephanienadalincantirecom investor karen keyes karenkeyescantirecom"

In [10]:
Question= "How does Canadian Tire improve customer satisfaction?"
answer = answer_question(Context, Question)
print(f"Question: {Question}")
print(f"Answer: {answer}")

Invalid span detected: Adjusting indices.
Question: How does Canadian Tire improve customer satisfaction?
Answer: canadian tire corporation report third quarter
